In [29]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.firefox.options import Options

In [35]:
options = Options()
options.headless = True
driver = webdriver.Firefox(options=options, executable_path=r'.\geckodriver.exe')
driver.get("http://google.com/")
print ("Headless Firefox Initialized")
driver.quit()

Headless Firefox Initialized


In [24]:
# scraping the job grouplists
rooturl = "https://www.nsp.cz/odborny-smer"
rootpage = requests.get(rooturl).text
soup = BeautifulSoup(rootpage, 'html.parser')
sel = soup.select(".bricklayer li a")
rootlinks = [a["href"] for a in sel]

In [27]:
# getting the links for individual jobs
#for link in rootlinks:
#    print(link)
print(requests.get(rootlinks[0]).text)

<!DOCTYPE html>
<html lang="cs">
<head>
  <meta charset="utf-8"/>
      <base href="https://www.nsp.cz/"/>
    <meta name="viewport" content="width=device-width, initial-scale=1"/>
  <link rel="shortcut icon" href="/favicon.ico"/>
  <meta name="description" content="Národní soustava povolání (NSP) je nástroj pro moderní řízení lidských zdrojů."/>
  <title>Bankovnictví, finance a pojišťovnictví - Odborné směry</title>
  <script id="nspLoadingTemplate" type="text/x-template">
    <div class="nspLoading">
  <div class="nspLoading1 nspLoading__child"></div>
  <div class="nspLoading2 nspLoading__child"></div>
  <div class="nspLoading3 nspLoading__child"></div>
  <div class="nspLoading4 nspLoading__child"></div>
  <div class="nspLoading5 nspLoading__child"></div>
  <div class="nspLoading6 nspLoading__child"></div>
  <div class="nspLoading7 nspLoading__child"></div>
  <div class="nspLoading8 nspLoading__child"></div>
  <div class="nspLoading9 nspLoading__child"></div>
  <div class="nspLoading